<h1>Hip roof</h1>

<h2>Simple reference model</h2>
![alt text](reference_model_simplex.gif)

<h2>Complex reference model</h2>
![alt text](reference_model_complex.jpg)

In [119]:
from pyplasm import *
verts_simple = [[0,0,0],[20,0,0],[20,10,0],[0,10,0],[5,5,3],[15,5,3],[5,5,0],[15,5,0]]
cells_simple = [[1,4,5,7],[1,2,5,6,7,8],[2,6,3,8],[3,4,5,6,7,8]]
verts_complex = [[0,10,0],[12,10,0],[12,0,0],[20,0,0],[16,4,0],[16,4,3],[16,15,0],[16,15,3],[20,20,0],[0,20,0],[5,15,0],[5,15,3]]
cells_complex = [[1,2,7,8,11,12],[1,11,12,10],[10,11,12,7,8,9],[2,3,5,6,7,8],[4,5,6,7,8,9],[3,4,6]]

In [120]:
def clean_lists(hpc):
    skel = SKEL_1(hpc)
    skeleton = OFFSET([.2,.2,.4])(skel)      
    skeleton2 = UKPOL(SKEL_2(hpc))    
    vert=skeleton2[0]
    cell=skeleton2[1]
    final=[]
    vertex=[]
    cells=[]
    vertex2=[]
    
    for c in cell:
        f=[]
        for v in c:
            f.append(vert[v-1])
        final.append(f)
    
    for u in final:
        for v in u:
            if v not in vertex:
                vertex.append(v)
            else:
                v=vertex.index(v)
    
    for u in final:
        x=[]
        for v in u:
            x.append(vertex.index(v)+1)
        cells.append(x)

    cont=0
    for v in vertex:
        if v[2]>0:
            vertex2.append(v)
        else:
            for v2 in vertex:
                if v2[2]>0:
                    if v2[0]==v[0] and v2[1]==v[1]:
                        vertex2.append(v2)
                        cont=1
            if not(cont):
                vertex2.append(v)
            cont=0
    
    return [vertex2,cells,skeleton]

In [121]:
def ggpl_hip_roof(verts, cells):
    hpc = MKPOL([verts,cells,None])
    lists = clean_lists(hpc)
    vertex2 = lists[0]
    cell = lists[1]
    skeleton = lists[2]
    coverMkp=MKPOL([vertex2,cell,None])
    coverStr=STRUCT([T(3)(.2),coverMkp])
    coverOff=OFFSET([.2,.2,.4])(coverStr)
    skeleton=COLOR(BLACK)(skeleton)
    roof= STRUCT([coverOff,skeleton]) 
    return roof

In [122]:
roof = ggpl_hip_roof(verts_simple,cells_simple)
VIEW(roof)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000B14FDE0> >

<h2>Final result (simple)</h2>
![alt text](hip_roof_simple.jpg)
![alt text](hip_roof_simple_under.jpg)

<h2>Final result (complex)</h2>
![alt text](skeleton_complex.jpg)
![alt text](hip_roof_complex.jpg)